In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
base_dir = os.getcwd()

## Load data 

In [14]:
data_filename = "m29_7arcmin_panstarrs_data.csv"
data = pd.read_csv(os.path.join(base_dir, data_filename), sep=';')
useful_columns = ["_r", "RAJ2000", "DEJ2000", "gmag", "e_gmag", "rmag", "e_rmag", "imag", "e_imag", "zmag", "e_zmag"]
data = data[useful_columns]

def to_float(x): 
    try: 
        return float(x)
    except Exception:
        return np.nan 
data = data.applymap(to_float).astype(float)

In [15]:
data.dropna(subset=["gmag", "rmag", "imag", "zmag"], inplace=True)
print(data.describe())

                _r      RAJ2000      DEJ2000         gmag       e_gmag  \
count  2641.000000  2641.000000  2641.000000  2641.000000  1629.000000   
mean      4.725604   305.935539    38.483476    21.623468     0.055435   
std       1.592680     0.075223     0.058280     3.276687     0.065481   
min       0.034900   305.794028    38.371720     8.725000     0.000000   
25%       3.592300   305.873864    38.435752    19.892500     0.009100   
50%       4.972200   305.929754    38.482309    21.618000     0.028400   
75%       6.075000   305.994990    38.529979    23.485000     0.070900   
max       6.999800   306.091144    38.602637    30.090000     0.432200   

              rmag       e_rmag         imag       e_imag         zmag  \
count  2641.000000  2131.000000  2641.000000  2451.000000  2641.000000   
mean     19.719195     0.036523    18.562242     0.022226    17.803363   
std       2.478741     0.049236     2.039345     0.034142     1.803760   
min       8.338800     0.000000     8

In [16]:
def ugriz_to_ubvri(ugriz):
    u, g, r, i, z = ugriz
    u = np.array(u)
    g = np.array(g)
    r = np.array(r)
    i = np.array(i)
    z = np.array(z)

    V = g - 0.59*(g - r) - 0.01
    B = g + 0.39*(g - r) + 0.21
    R = V - (1.09*(r - i) + 0.22)
    U = 0.77*(u - g) - 0.88 + B

    I = R - (r - i + 0.21)
    return (U, B, V, R, I)

gmags = data["gmag"]
umags = pd.Series([69 for _ in range(len(gmags))])
rmags = data["rmag"]
imags = data["imag"]
zmags = data["zmag"]
_, data["B"], data["V"], data["R"], data["I"] = ugriz_to_ubvri((umags, gmags, rmags, imags, zmags))

## Apply distance correction, reddening

In [17]:
m29_dist = 1148 # pc
obs_minus_absolue = 5*np.log10(m29_dist) - 5
data = data[data["V"] < 9 + obs_minus_absolue]
data["B"] -= obs_minus_absolue
data["V"] -= obs_minus_absolue
data["R"] -= obs_minus_absolue
data["I"] -= obs_minus_absolue
a_redd = 0.744

A_v = 3.1*a_redd
A_b = 1.32*A_v
A_r = 0.79*A_v
A_i = 0.55*A_v

data["B"] -= A_b
data["V"] -= A_v 
data["R"] -= A_r
data["I"] -= A_i

## Load isochrones

In [38]:
times = [6.0 + i*0.1 for i in range(31)]

filenames = [f"isochrone_{10*t:.0f}.dat" for t in times]
isochrone_cols = ["M_ini", "MV", "U-B", "B-V", "V-R", "V-I"]
isochrones = {
    times[i]: pd.read_csv(os.path.join("..", "isochrones", filenames[i]), delim_whitespace=True)[isochrone_cols] for i in range(len(filenames))
}


In [47]:
%matplotlib qt
from matplotlib import colors

fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3)
B = data["B"]
V = data["V"]
R = data["R"]
I = data["I"]

# ax1 B-V vs V
ax1.scatter(B-V, V, s=1)
ax1.set_xlabel("B-V")
ax1.set_ylabel("V")

# ax2 V-R vs R
ax2.scatter(V-R, R, s=1)
ax2.set_xlabel("V-R")
ax2.set_ylabel("R")

# ax3 R-I vs I
ax3.scatter(R-I, I, s=1)
ax3.set_xlabel("R-I")
ax3.set_ylabel("I")

plt.tight_layout()
# isochrone
cdict = {'red':   ((0.0,  0.22, 0.0),
                   (0.5,  1.0, 1.0),
                   (1.0,  0.89, 1.0)),

         'green': ((0.0,  0.49, 0.0),
                   (0.5,  1.0, 1.0),
                   (1.0,  0.12, 1.0)),

         'blue':  ((0.0,  0.72, 0.0),
                   (0.5,  0.0, 0.0),
                   (1.0,  0.11, 1.0))}

cmap = colors.LinearSegmentedColormap('custom', cdict)

times = [6.0 + 0.1*i for i in range(31)]
for index, log_t in enumerate(times):
    vmags = isochrones[log_t]["MV"] 
    bmags = isochrones[log_t]["B-V"] + vmags
    rmags = vmags - isochrones[log_t]["V-R"]
    imags = vmags - isochrones[log_t]["V-I"]
    ax1.plot(bmags - vmags, bmags, c=cmap(index/len(times)))
    ax2.plot(vmags - rmags, vmags, c=cmap(index/len(times)))
    ax3.plot(rmags - imags, rmags, c=cmap(index/len(times)))
ax1.set_xlim((-0.5, 2.5))
ax1.set_ylim((-2, 7.5))

ax2.set_xlim((-0.5, 2))
ax2.set_ylim((-2, 7))

ax3.set_ylim((-5, 6.5))
ax3.set_xlim((-1, 1.5))
ax1.invert_yaxis()
ax2.invert_yaxis()
ax3.invert_yaxis()